In [80]:
import os
import json
import pandas as pd
from pathlib import Path
import numpy as np

In [49]:
root_dir = Path('/h/suo/dev/tree-search-planning/muzero-general/out')

In [59]:
evaluations = []
for env in os.listdir(root_dir):
    for model in os.listdir(root_dir / env):
        for run in os.listdir(root_dir / env / model):
            try:
                stats_json = next((root_dir / env / model / run).glob('*.stats.json'))
                evaluations.append([env, model, run, stats_json])
            except:
                pass

In [67]:
evaluations_df = pd.DataFrame(evaluations, columns=['env', 'model', 'run', 'stats_json'])
evaluations_df.head()

,env,model,run,stats_json
0,HighwayEnv,MCTSAgent,run_20210316-153909_25172,/h/suo/dev/tree-search-planning/muzero-general...
1,HighwayEnv,MCTSAgent,run_20210316-152933_22463,/h/suo/dev/tree-search-planning/muzero-general...
2,HighwayEnv,DQNAgent,run_20210316-145415_10891,/h/suo/dev/tree-search-planning/muzero-general...
3,HighwayEnv,DQNAgent,run_20210316-145336_10605,/h/suo/dev/tree-search-planning/muzero-general...
4,HighwayEnv,DQNAgent,run_20210316-181851_10244,/h/suo/dev/tree-search-planning/muzero-general...


In [94]:
def get_stats(run_dir):
    with open(stats_json) as f:
        stats = json.load(f)
    return stats

In [95]:
evaluations_df['stats'] = evaluations_df['stats_json'].apply(get_stats)
evaluations_df.head()

,env,model,run,stats_json,stats,mean_episode_length,mean_episode_reward
0,HighwayEnv,MCTSAgent,run_20210316-153909_25172,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165
1,HighwayEnv,MCTSAgent,run_20210316-152933_22463,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165
2,HighwayEnv,DQNAgent,run_20210316-145415_10891,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165
3,HighwayEnv,DQNAgent,run_20210316-145336_10605,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165
4,HighwayEnv,DQNAgent,run_20210316-181851_10244,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165


In [96]:
def mean_episode_length(stat_dict):
    return np.mean(stat_dict['episode_lengths'])
def mean_total_reward(stat_dict):
    return np.mean(stat_dict['episode_rewards'])

In [97]:
evaluations_df['mean_episode_length'] = evaluations_df['stats'].apply(mean_episode_length)
evaluations_df['mean_episode_reward'] = evaluations_df['stats'].apply(mean_total_reward)

In [98]:
evaluations_df

,env,model,run,stats_json,stats,mean_episode_length,mean_episode_reward
0,HighwayEnv,MCTSAgent,run_20210316-153909_25172,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165
1,HighwayEnv,MCTSAgent,run_20210316-152933_22463,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165
2,HighwayEnv,DQNAgent,run_20210316-145415_10891,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165
3,HighwayEnv,DQNAgent,run_20210316-145336_10605,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165
4,HighwayEnv,DQNAgent,run_20210316-181851_10244,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165
5,HighwayEnv,DQNAgent,run_20210316-181404_8647,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165
6,HighwayEnv,DQNAgent,run_20210316-181828_10133,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165
7,HighwayEnv,DQNAgent,run_20210408-052910_18495,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165
8,HighwayEnv,DQNAgent,run_20210316-181429_8918,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165
9,HighwayEnv,DQNAgent,run_20210408-053122_19480,/h/suo/dev/tree-search-planning/muzero-general...,"{'initial_reset_timestamp': 1617879186.096059,...",9.2,6.944165
